In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [2]:
model = MobileNetV2(weights = "imagenet",
                    include_top = False,
                    input_shape = (200, 200, 3)
                   )

In [3]:
for (i, layer) in enumerate(model.layers):
    print(f"{i} {layer.__class__.__name__} {layer.trainable}")

0 InputLayer True
1 Conv2D True
2 BatchNormalization True
3 ReLU True
4 DepthwiseConv2D True
5 BatchNormalization True
6 ReLU True
7 Conv2D True
8 BatchNormalization True
9 Conv2D True
10 BatchNormalization True
11 ReLU True
12 ZeroPadding2D True
13 DepthwiseConv2D True
14 BatchNormalization True
15 ReLU True
16 Conv2D True
17 BatchNormalization True
18 Conv2D True
19 BatchNormalization True
20 ReLU True
21 DepthwiseConv2D True
22 BatchNormalization True
23 ReLU True
24 Conv2D True
25 BatchNormalization True
26 Add True
27 Conv2D True
28 BatchNormalization True
29 ReLU True
30 ZeroPadding2D True
31 DepthwiseConv2D True
32 BatchNormalization True
33 ReLU True
34 Conv2D True
35 BatchNormalization True
36 Conv2D True
37 BatchNormalization True
38 ReLU True
39 DepthwiseConv2D True
40 BatchNormalization True
41 ReLU True
42 Conv2D True
43 BatchNormalization True
44 Add True
45 Conv2D True
46 BatchNormalization True
47 ReLU True
48 DepthwiseConv2D True
49 BatchNormalization True
50 ReLU True

In [4]:
for layer in model.layers:
    layer.trainable = False

In [5]:
for (i, layer) in enumerate(model.layers):
    print(f"{i} {layer.__class__.__name__} {layer.trainable}")

0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 Conv2D False
10 BatchNormalization False
11 ReLU False
12 ZeroPadding2D False
13 DepthwiseConv2D False
14 BatchNormalization False
15 ReLU False
16 Conv2D False
17 BatchNormalization False
18 Conv2D False
19 BatchNormalization False
20 ReLU False
21 DepthwiseConv2D False
22 BatchNormalization False
23 ReLU False
24 Conv2D False
25 BatchNormalization False
26 Add False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 ZeroPadding2D False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 Conv2D False
37 BatchNormalization False
38 ReLU False
39 DepthwiseConv2D False
40 BatchNormalization False
41 ReLU False
42 Conv2D False
43 BatchNormalization False
44 Add False
45 Conv2D False
46 BatchNormalization False
47 ReLU False
48 DepthwiseCon

In [6]:
def add_layer_at_bottom(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(512, activation = "relu")(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_data_dir = "data/train"
val_data_dir = "data/test"

In [9]:
from keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 45,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   horizontal_flip = True,
                                   fill_mode = "nearest")

val_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (200, 200),
                                                    batch_size = 32,
                                                    class_mode = "categorical")

val_generator = val_datagen.flow_from_directory(val_data_dir,
                                                    target_size = (200, 200),
                                                    batch_size = 32,
                                                    class_mode = "categorical")

Found 140 images belonging to 4 classes.
Found 20 images belonging to 4 classes.


In [11]:
train_class_names = set()
num_train_samples = 0

for i in train_generator.filenames:
    train_class_names.add(i.split('\\')[0])
    num_train_samples += 1
print(num_train_samples)
train_class_names

140


{'oval', 'rectangle', 'stickman', 'triangle'}

In [12]:
val_class_names = set()
num_val_samples = 0

for i in val_generator.filenames:
    val_class_names.add(i.split('\\')[0])
    num_val_samples += 1
print(num_val_samples)
val_class_names

20


{'oval', 'rectangle', 'stickman', 'triangle'}

In [13]:
num_classes = len(train_generator.class_indices)

FC_head = add_layer_at_bottom(model, num_classes)

main_model = Model(inputs = model.input,
                   outputs = FC_head)

main_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 100, 100, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 100, 100, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [14]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [15]:
checkpoint = ModelCheckpoint("Shape_recognition.h5",
                             monitor = "loss",
                             mode = "min",
                             save_best_only = True, verbose = 1)

earlystop = EarlyStopping(monitor = "loss",
                          min_delta = 0,
                          patience = 50,
                          verbose = 1, 
                          restore_best_weights = True)

callbacks = [checkpoint, earlystop]

main_model.compile(loss = "categorical_crossentropy",
                   optimizer = RMSprop(learning_rate = 0.001),
                   metrics = ["accuracy"])

epochs = 50
batch_size = 32

history = main_model.fit(train_generator,
                         steps_per_epoch = num_train_samples // batch_size,
                         epochs = epochs,
                         callbacks = callbacks,
                         validation_data = val_generator,
                         validation_steps = num_val_samples // batch_size)

Epoch 1/50
4/4 [==============================] - ETA: 0s - loss: 10.2208 - accuracy: 0.2685
Epoch 1: loss improved from inf to 10.22083, saving model to Shape_recognition.h5
4/4 [==============================] - 4s 381ms/step - loss: 10.2208 - accuracy: 0.2685
Epoch 2/50
4/4 [==============================] - ETA: 0s - loss: 1.2136 - accuracy: 0.5370
Epoch 2: loss improved from 10.22083 to 1.21363, saving model to Shape_recognition.h5
4/4 [==============================] - 1s 357ms/step - loss: 1.2136 - accuracy: 0.5370
Epoch 3/50
4/4 [==============================] - ETA: 0s - loss: 0.3941 - accuracy: 0.8148
Epoch 3: loss improved from 1.21363 to 0.39412, saving model to Shape_recognition.h5
4/4 [==============================] - 1s 443ms/step - loss: 0.3941 - accuracy: 0.8148
Epoch 4/50
4/4 [==============================] - ETA: 0s - loss: 0.2706 - accuracy: 0.8796
Epoch 4: loss improved from 0.39412 to 0.27056, saving model to Shape_recognition.h5
4/4 [==========================

In [16]:
import sys
!{sys.executable} -m pip install opencv-python

In [17]:
import cv2

In [18]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from PIL import Image

test_image = "data/validation/stickman/001.jpg"

# Load image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image_to_tensor = tf.convert_to_tensor(np.array(image))
    return image, tf.expand_dims(image_to_tensor, axis = 0)



preprocess_image(test_image)

# Predict using the model
def predict(model, image_tensor):
    res = model.predict(image_tensor)
    model.evaluate(res)
    with tf.no_gradient():
        outputs = model(image_tensor)
        probabilities = tf.nn.softmax(outputs, axis = 1)
    return probabilities.cpu().numpy().flatten()


# Visualization method
def visualize_predictions(image, probabilities, class_names):
    fig, axarr = plt.subplots(1, 2, figsize = (14, 7))

    # Display image
    axarr[0].imshow(img)
    axarr[0].axis("off")

    # Display predictions
    axarr[1].barh(class_names, probabilities)
    axarr[1].set_xlabel("Probability")
    axarr[1].set_title("Class Predictions")
    axarr[1].set_xlim(0, 1)

    plt.tight_layout()

    plt.show()

# Class names
classes = ["Oval", "Rectangle", "Stickman", "Triangle"]

# img = cv2.imread("data/validation/stickman/005.jpg")
# plt.imshow(img)
# img = img.reshape(1, 200, 200, 3)
# res = main_model.predict(img)
# out[np.argmax(res)]

original_image, image_tensor = preprocess_image(test_image)
print(predict(main_model, image_tensor))

# probabilities = predict(main_model(input_shape = (200, 200, 3), image_tensor)

# visualize_predictions(original_image, probabilities, classes)

1/1 [==============================] - 1s 734ms/step


ValueError: in user code:

    File "C:\Users\Chris\miniconda3\envs\core\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\Chris\miniconda3\envs\core\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Chris\miniconda3\envs\core\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\Chris\miniconda3\envs\core\lib\site-packages\keras\engine\training.py", line 1665, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\Chris\miniconda3\envs\core\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Chris\miniconda3\envs\core\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_c

In [19]:
history.history??

Type:        dict
String form: {'loss': [10.220826148986816, 1.2136305570602417, 0.39411869645118713, 0.2705601155757904, 0.21037301421165466, 0.06552980095148087, 0.5853488445281982, 0.13581886887550354, 0.19756267964839935, 0.1260077953338623, 0.15543457865715027, 0.270478218793869, 0.08144499361515045, 0.2673841118812561, 0.19431203603744507, 0.20352140069007874, 0.739203691482544, 0.15722663700580597, 0.1696263551712036, 0.21093714237213135, 0.22907043993473053, 0.04779376462101936, 0.1368158757686615, 0.18802303075790405, 0.3316175043582916, 0.18769870698451996, 0.16724614799022675, 0.4736417233943939, 0.07621007412672043, 0.05656179413199425, 0.11056617647409439, 0.5354960560798645, 0.11113449186086655, 0.29879727959632874, 0.0795845165848732, 0.06096518412232399, 0.14953932166099548, 0.16976779699325562, 0.11037415266036987, 0.20818611979484558, 0.30325639247894287, 0.027643993496894836, 0.08518331497907639, 0.17947949469089508, 0.38437420129776, 0.061682283878326416, 0.17820680